##### The code here is taken from Copernicus Climate Change website to get netCDF datafile to retrieve Climate data from *Copernicus Climate Change Server*

##### Requirement (Windows with Anaconda VIrtual Environment)  
- Create an account here: https://cds.climate.copernicus.eu
- Install cdsapi via conda forge
- Install netCDF4 via conda forge 
- Go to the directory: C://Users//Username and create a file name .cdsapirc
- Copy the URL and Key form here and paste to the file created above: https://cds.climate.copernicus.eu/api-how-to
- Choose the data you need from here: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=form

In [1]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format': 'netcdf',
        'product_type': 'monthly_averaged_reanalysis_by_hour_of_day',
        'variable': [
            '2m_dewpoint_temperature', '2m_temperature',
        ],
        'year': [
            '2018', '2019',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'time': [
            '05:00', '08:00', '12:00',
            '13:00', '19:00',
        ],
    },
    'temperature.nc')

2019-12-27 18:38:13,428 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2019-12-27 18:38:15,435 INFO Request is queued
2019-12-27 18:38:16,689 INFO Request is running
2019-12-27 18:39:32,913 INFO Request is completed
2019-12-27 18:39:32,914 INFO Downloading http://136.156.133.42/cache-compute-0014/cache/data1/adaptor.mars.internal-1577452145.2320635-3171-33-a9e4b88d-a41a-4b9f-891c-b2ce265f2ee1.nc to temperature.nc (910.9M)
2019-12-27 18:43:21,855 INFO Download rate 4M/s                                                                        


Result(content_length=955191712,content_type=application/x-netcdf,location=http://136.156.133.42/cache-compute-0014/cache/data1/adaptor.mars.internal-1577452145.2320635-3171-33-a9e4b88d-a41a-4b9f-891c-b2ce265f2ee1.nc)

In [2]:
import xarray as xr

ds = xr.open_dataset('temperature.nc')
df = ds.to_dataframe()

In [41]:
len(ds['latitude'])

721

In [12]:
x = df.to_csv('tem-csv.csv')

In [27]:
#learn about the dataset here
#https://confluence.ecmwf.int/display/CKB/ERA5-Land+data+documentation#ERA5-Landdatadocumentation-Dataformat

## Extracting temperature for gujarat region

In [2]:
import pandas as pd

In [59]:
#Holy-Moly!! RAM is too low to read a 6GB csv file directly, so reading in chunk
df_chunk = pd.read_csv('tem-csv.csv', chunksize=100000)

In [60]:
#drop useless data. We are interested in 'some' coordinates only
chunk_list = []
for chunk in df_chunk:
    x = chunk[chunk['latitude']<24]
    x = x[x['longitude']<73]
    chunk_list.append(x)
    
df_concat = pd.concat(chunk_list)

In [62]:
df_concat.to_csv('temperature-filtered.csv')

In [3]:
temp = pd.read_csv('temperature-filtered.csv')

In [9]:
temp

,Unnamed: 0,latitude,longitude,time,d2m_0001,t2m_0001,d2m_0005,t2m_0005
0,43884000,23.75,0.00,2018-01-01 05:00:00,269.56660,283.93658,NaN,NaN
1,43884001,23.75,0.00,2018-01-01 08:00:00,270.60030,285.45166,NaN,NaN
2,43884002,23.75,0.00,2018-01-01 12:00:00,270.05487,295.57895,NaN,NaN
3,43884003,23.75,0.00,2018-01-01 13:00:00,269.61295,296.64612,NaN,NaN
4,43884004,23.75,0.00,2018-01-01 19:00:00,269.44608,293.79440,NaN,NaN
...,...,...,...,...,...,...,...,...
15312475,119265575,-90.00,72.75,2019-11-01 05:00:00,NaN,NaN,234.21346,238.25389
15312476,119265576,-90.00,72.75,2019-11-01 08:00:00,NaN,NaN,234.10092,238.22862
15312477,119265577,-90.00,72.75,2019-11-01 12:00:00,NaN,NaN,234.47913,238.62624
15312478,119265578,-90.00,72.75,2019-11-01 13:00:00,NaN,NaN,234.48608,238.63954


In [10]:
tempx = temp[ (temp['latitude']>23) & (temp['latitude']<24) ]

In [12]:
tempx = temp[ (temp['longitude']>72) & (temp['longitude']<73)]

In [15]:
tempx.index = pd.to_datetime(tempx.time)

In [17]:
tempxr = tempx['2019-09-01':'2019-11-30']

In [22]:
tempxr['t2m_0001'] = tempxr['t2m_0001']-273.15

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
tempxr['d2m_0001'] = tempxr['d2m_0001']-273.15

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
tempxr.to_csv('gujarat-temp-2019.csv')

In [27]:
tempxr

,Unnamed: 0,latitude,longitude,time,d2m_0001,t2m_0001,d2m_0005,t2m_0005
time,,,,,,,,
2019-09-01 05:00:00,43917335,23.75,72.25,2019-09-01 05:00:00,25.71900,28.91400,NaN,NaN
2019-09-01 08:00:00,43917336,23.75,72.25,2019-09-01 08:00:00,25.67263,31.30886,NaN,NaN
2019-09-01 12:00:00,43917337,23.75,72.25,2019-09-01 12:00:00,25.96005,30.51650,NaN,NaN
2019-09-01 13:00:00,43917338,23.75,72.25,2019-09-01 13:00:00,25.97700,30.18636,NaN,NaN
2019-09-01 19:00:00,43917339,23.75,72.25,2019-09-01 19:00:00,25.34197,27.38284,NaN,NaN
...,...,...,...,...,...,...,...,...
2019-11-01 05:00:00,119265575,-90.00,72.75,2019-11-01 05:00:00,NaN,NaN,234.21346,238.25389
2019-11-01 08:00:00,119265576,-90.00,72.75,2019-11-01 08:00:00,NaN,NaN,234.10092,238.22862
2019-11-01 12:00:00,119265577,-90.00,72.75,2019-11-01 12:00:00,NaN,NaN,234.47913,238.62624
